In [1]:
from google.colab import files
uploaded = files.upload()

Saving FlightDelays.csv to FlightDelays.csv


In [2]:
import pandas as pd

In [3]:
df = pd.read_csv('FlightDelays.csv')

In [4]:
rvar_list =['Weather']
df_sample1 = df.drop(columns=rvar_list)
cvar_list = ['Binned_CRS_DEP_TIME','CARRIER','DEST','ORIGIN','DAY_WEEK'
,'Flight Status']

In [5]:
df_sample2 = df_sample1.copy()
df_sample2[cvar_list] = df_sample1[cvar_list].astype('category')

df_sample3 = df_sample2.copy()
df_sample3 = pd.get_dummies(df_sample2, prefix_sep='_')

rdummies = ['Binned_CRS_DEP_TIME_6','CARRIER_DH', 'DEST_LGA', 'DAY_WEEK_5', 'Flight Status_Delayed']

df_sample4 = df_sample3.copy()
df_sample4 = df_sample3.drop(columns=rdummies)

print(df_sample4.columns.values)

print(df_sample4)

['Binned_CRS_DEP_TIME_1' 'Binned_CRS_DEP_TIME_2' 'Binned_CRS_DEP_TIME_3'
 'Binned_CRS_DEP_TIME_4' 'Binned_CRS_DEP_TIME_5' 'Binned_CRS_DEP_TIME_7'
 'Binned_CRS_DEP_TIME_8' 'CARRIER_CO' 'CARRIER_DL' 'CARRIER_MQ'
 'CARRIER_OH' 'CARRIER_RU' 'CARRIER_UA' 'CARRIER_US' 'DEST_EWR' 'DEST_JFK'
 'ORIGIN_BWI' 'ORIGIN_DCA' 'ORIGIN_IAD' 'DAY_WEEK_1' 'DAY_WEEK_2'
 'DAY_WEEK_3' 'DAY_WEEK_4' 'DAY_WEEK_6' 'DAY_WEEK_7'
 'Flight Status_On-time']
      Binned_CRS_DEP_TIME_1  ...  Flight Status_On-time
0                         0  ...                      0
1                         0  ...                      0
2                         0  ...                      0
3                         0  ...                      0
4                         0  ...                      0
...                     ...  ...                    ...
1314                      1  ...                      1
1315                      0  ...                      1
1316                      0  ...                      1
1317      

In [6]:
from sklearn.model_selection import train_test_split

df4partition = df_sample4
testpart_size = 0.2

df_nontestData, df_testData = train_test_split(df4partition, test_size=testpart_size, random_state=1)

print(df_nontestData)

      Binned_CRS_DEP_TIME_1  ...  Flight Status_On-time
710                       0  ...                      1
1258                      0  ...                      1
435                       0  ...                      1
987                       0  ...                      0
1286                      0  ...                      1
...                     ...  ...                    ...
715                       0  ...                      1
905                       0  ...                      1
1096                      0  ...                      0
235                       0  ...                      0
1061                      0  ...                      0

[1055 rows x 26 columns]


In [7]:
from sklearn.linear_model import LogisticRegression, LogisticRegressionCV
DV = 'Flight Status_On-time'
y = df_nontestData[DV]
X = df_nontestData.drop(columns=[DV])

kfolds = 5

min_alpha = 0.001
max_alpha = 100

n_candidates = 1000

import numpy as np

alpha_list = list(np.linspace(min_alpha, max_alpha, num=n_candidates))

C_list = list(1/np.linspace(min_alpha, max_alpha, num=n_candidates))

clf_optimal = LogisticRegressionCV(Cs=C_list, cv=kfolds, scoring='roc_auc', penalty='l1', solver='saga', max_iter=200, random_state=1, n_jobs=-1).fit(X,y)

print(1/clf_optimal.C_)

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_sag.py:354: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  ConvergenceWarning,


[0.4013964]


In [8]:
y_test_actual = df_testData[DV]

X_test = df_testData.drop(columns=[DV])


from sklearn.metrics import roc_auc_score
print(roc_auc_score(y_test_actual, clf_optimal.predict_proba(X_test)[:,1]))

0.7040675844806008
